In [ ]:
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
%matplotlib inline

# Feedback vs. no feedback

In [ ]:
from analysis.scripts.mbi import mbi_data_general_fits as fits; reload(fits)
####
#parameters / timestamps
ssro_tstamp = '20160817_094252'
tstamp_feedback = '20160817_141641'
tstamp_no_feedback = '20160817_132726'
####
Result_w_feedback = fits.exp_sin(timestamp=  tstamp_feedback,
                      offset=[0.5], amplitude = [0.5], center = [0], decay_constant = [300], 
                      exp_power = [2],
                      frequency = [0.], phase =[0], 
                      fixed = [0,2,4,5,6],
                      ylim = None,
                      plot_fit = True, do_print = True, show_guess = False)[0]
Result_wo_feedback = fits.exp_sin(timestamp=  tstamp_no_feedback,
                      offset=[0.5], amplitude = [0.5], center = [0], decay_constant = [300], 
                      exp_power = [2],
                      frequency = [0.04], phase =[0], 
                      fixed = [0,2,4],
                      ylim = None,
                      plot_fit = True, do_print = True, show_guess = False)[0]

In [ ]:
save_folder = r'K:\ns\qt\Diamond\Projects\Purification\Paper\Plots' 

fig = plt.figure(figsize =(0.7577*6,0.7577*4))
ax = plt.subplot(111)
x_wo_feed = np.linspace(Result_wo_feedback['x'][0],Result_wo_feedback['x'][-1],200)
x_w_feed = np.linspace(Result_w_feedback['x'][0],Result_w_feedback['x'][-1],200)

### formatting
lw = 1.5
fontsize = 12
markersize = 5

col_w_feed = '#50A53A'
col_wo_feed = 'black'

ax.set_ylabel('p(ms = 0)',size = fontsize)
ax.set_xlabel('N',size = fontsize)
ax.set_xticklabels([0,20,40,60,80,100],size = fontsize)
ax.set_yticklabels([0.0,0.2,0.4,0.6,0.8,1.0],size = fontsize)
ax.xaxis.set_tick_params(width=lw)
ax.yaxis.set_tick_params(width=lw)
[i.set_linewidth(lw) for i in ax.spines.itervalues()]

### commencde plotting
plt.plot(x_w_feed,Result_w_feedback['fitfunc'](x_w_feed),color = col_w_feed,lw=lw)
plt.plot(x_wo_feed,Result_wo_feedback['fitfunc'](x_wo_feed), color = col_wo_feed,lw = lw)
plt.errorbar(Result_wo_feedback['x'],Result_wo_feedback['y'],Result_wo_feedback['y_u'],fmt = 'o',color = col_wo_feed,ms = markersize)
plt.errorbar(Result_w_feedback['x'],Result_w_feedback['y'],Result_w_feedback['y_u'],fmt = 'o',color = col_w_feed,ms = markersize)
plt.savefig(os.path.join(save_folder,'feedback_nofeedback.png'),format='png',bbox_inches = 'tight',pad_inches=0.3)
plt.savefig(os.path.join(save_folder,'feedback_nofeedback.p'),format='png',bbox_inches = 'tight',pad_inches=0.3)

In [ ]:
ax.set_ylabel?

In [ ]:
plt.figure?

In [ ]:
plt.errorbar?